## Ajuste de Dados
Notebook para ajuste dos dados de planilhas de OD oxímetro e MINIPAR

O programa faz:

1- Leitura dos dados da Sonda de OD;

2- Ajustes dos dados de hora (a planilha apresenta dados em segundos que serão passados para DD/MM/YYYY H:M);

3- Calcula a saturação de OD em dada temperatura;

4- Retira dados de OD acima da saturação;  

5- Reorganiza os dados faltantes em dados em Branco; 


Autor: 
* MSc. Eng. Pedro Gabriel Grochocki Gabriel $^\text{a}$

$^\text{a}$ Curso de Doutorado em Engenharia Ambiental - UFPR<br/>

email - pedro.gmlivre@gmail.com

Whatsapp - (41) 99810-5797

## alterações:

23/10/2023 - Adicionado dados miniPAR

In [4]:
# importa bibliotecas
import numpy as np
import pandas as pd   
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import rc
import matplotlib
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import datetime
from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange
import matplotlib.dates as mdates
from IPython.display import display, Markdown
from datetime import date
from datetime import datetime as dt
from pathlib import Path
from datetime import timedelta

import os
import subprocess
import datetime
from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange
import matplotlib.dates as mdates
from IPython.display import display, Markdown
from bokeh.plotting import figure, output_file, show
import re
from IPython.display import display
now = datetime.datetime.now()
now = datetime.datetime.strftime(now, '%Y-%m-%dT %H:%M')

In [5]:
##########################################################################
##########################################################################
def arruma_OD(ARQ):
    ler = open(ARQ + ".txt", "r")
    N = ler.readlines()
    
    ler.close()
    y = ARQ[31:]
    x = ARQ[:-18]
    #print(x + "AUX/" + y +".aux")
    esc = open(x + "AUX/" + y +".aux", "w")
    esc.write('Segundos,Temperatura(deg C),OD(mg/L)\n')
    for I in range(len(N)):
        A = re.split(r"[,+ :, +]", N[I])      
        if len(A) == 10:
            esc.write("{0},{1},{2}\n".format(A[0], A[4], A[7]))

    esc.close()
##########################################################################
##########################################################################

In [6]:
caminho = "Dados_FOTOAGUA/Dados_brutos/"
arqs=[
    caminho + 'LR',
    caminho + 'PV1'
]

## O código a seguir faz a união das planilhas por ano, pegando apenas os dados de Segundos, Temperatura e OD.

### A coluna "Data" é a data fornecida pelo nome da planilha.

In [7]:
for n in range(len(arqs)):
    print("Pasta acessada: ",arqs[n])
    Arquivo = arqs[n]
    NOME = Arquivo[28:]
    print("Tipo analisado: ", NOME)
    ##################################################################################
    (Path('.') / 'Dados Tratados/').mkdir(parents=True, exist_ok=True)# cria pasta para o Ano analisado
    (Path('.') / Arquivo /"AUX").mkdir(parents=True, exist_ok=True)# cria pasta para o Ano analisado
    (Path('.') / Arquivo /"UNIAO").mkdir(parents=True, exist_ok=True)# cria pasta para o Ano analisado
    super_caminhos = []
    for item in os.listdir(arqs[n]):
        if not item.startswith('.') and os.path.isfile(os.path.join(arqs[n], item)):
            super_caminhos.append(item)
    arquivos_lidos = super_caminhos
    print("Quantidade de arquivos: ",len(arquivos_lidos))  
    
    
    result_OD = pd.DataFrame()
    i = 0
    for k in range(len(arquivos_lidos)):
        nome = arquivos_lidos[k]
        nome = nome[:-4]
        #print(arqs[n]+"/"+nome)
        arruma_OD(arqs[n]+"/"+nome)
        dt_OD = pd.read_csv(arqs[n]+"/AUX/" +nome + ".aux", sep=',');
        nome = nome[:-7]
        dt_OD["Data"] = nome
    

        result_OD = pd.concat([result_OD, dt_OD])
    print(len(result_OD),"\n")
    result_OD = result_OD.sort_values(by=['Segundos'])
    ##################################################################################
    result_OD.to_csv(Arquivo + "/UNIAO/"+ NOME +".csv",index=False)

Pasta acessada:  Dados_FOTOAGUA/Dados_brutos/LR
Tipo analisado:  LR
Quantidade de arquivos:  385
108522 

Pasta acessada:  Dados_FOTOAGUA/Dados_brutos/PV1
Tipo analisado:  PV1
Quantidade de arquivos:  316
90098 



O código a seguir pega cada uma das planilhas geradas anteriormente e calcula a data com base nos segundos.

- O códigon separa os dados de dia, hora e minuto com base nesses valores de segundos. O formato dos dados de saída é :'YYYY-mm-dd HH:MM'

# Precisamos descobrir se iremos somar ou subtrair 3h (Estou mantendo a hora do sensor pois bate com os dados da planilha de campo)

In [8]:
for n in range(len(arqs)):
    print("Pasta acessada: ",arqs[n])
    Arquivo = arqs[n]
    NOME = Arquivo[28:]
    print("Tipo analisado: ", NOME)
    result_OD = pd.read_csv(Arquivo + "/UNIAO/"+ NOME +".csv",sep=',');
    ################################################################################## 
    result_OD = result_OD.sort_values(by=['Segundos'])
    #result_OD = result_OD.set_index('Data')
    cal = 0
    segundos = np.array(result_OD['Segundos'])
    segundos_ajustados = segundos - cal
    vetor_dia = np.zeros(len(result_OD))
    vetor_min = np.zeros(len(result_OD))
    vetor_hora = np.zeros(len(result_OD))
    vetor_ano = np.zeros(len(result_OD))
    vetor_month = np.zeros(len(result_OD))

    lista = []

    for k in range(len(result_OD)):
        objeto_data = dt.fromtimestamp(segundos_ajustados[k])
        data_br = objeto_data + timedelta(hours=0) # sem alterar a hora
        vetor_dia[k] = str(data_br.day)
        vetor_month[k] = str(data_br.month)
        vetor_ano[k] = str(data_br.year)
        vetor_hora[k] = str(data_br.hour)
        vetor_min[k] = str(data_br.minute)
        texto = str(data_br.year) + "-" + str(data_br.month) + "-" + str(data_br.day) + "  " + str(data_br.hour) + ":" + str(data_br.minute)
        lista.append(texto)
    result_OD["Data Calculada"] = lista   
    now = datetime.datetime.now()
    now = datetime.datetime.strftime(now, '%Y-%m-%dT %H:%M')
    now = datetime.datetime.now()
    now = datetime.datetime.strftime(now, '%Y-%m-%dT %H:%M')
    result_OD = result_OD.set_index('Data Calculada')
    ################################################################################## 
    result_OD.index = pd.to_datetime(result_OD.index)
    result_OD = result_OD.resample('10min').mean()
    result_OD.to_csv(Arquivo + "/UNIAO/Ajuste_hora_"+ NOME +".csv") 

Pasta acessada:  Dados_FOTOAGUA/Dados_brutos/LR
Tipo analisado:  LR


/var/folders/b1/jt5k1wqd6456ym1lhh704g000000gn/T/ipykernel_4817/1963794218.py:39: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  result_OD = result_OD.resample('10min').mean()


Pasta acessada:  Dados_FOTOAGUA/Dados_brutos/PV1
Tipo analisado:  PV1


/var/folders/b1/jt5k1wqd6456ym1lhh704g000000gn/T/ipykernel_4817/1963794218.py:39: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  result_OD = result_OD.resample('10min').mean()


# Cálculo de OD de Saturação e %
$$
OD_{sat} = ( 14.62 - 0.3898 T + 0.006969 T^{2} - 0.00005897 T^{3})(1-0.0000228675 L)^{5.167}
$$
com L = 904m

In [9]:
for n in range(len(arqs)):
    print("Pasta acessada: ",arqs[n])
    Arquivo = arqs[n]
    NOME = Arquivo[28:]
    print("Tipo analisado: ", NOME)
    result_OD = pd.read_csv(Arquivo + "/UNIAO/Ajuste_hora_"+ NOME +".csv",sep=',',index_col=0);
    ##################################################################################
    C_sat = np.zeros(len(result_OD))
    T = result_OD["Temperatura(deg C)"].values

    for n in range (len(result_OD)):
        C_sat[n] = (14.62 - 0.3898*T[n] +0.006969*T[n]*T[n] - 0.00005897*T[n]*T[n]*T[n])*(1-0.0000228675*904)**(5.167)
    Sat = np.zeros(len(result_OD)) # cria vetor para receber os dados de Saturação em %
    OD = result_OD["OD(mg/L)"] # recebe os dados de OD

    for n in range (len(result_OD)):
        Sat[n] = OD[n]*100/C_sat[n]
    result_OD["OD de Saturação (mg/L)"] = C_sat    
    result_OD["Saturação OD (%)"] = Sat
    ##################################################################################
    result_OD.to_csv(Arquivo + "/UNIAO/Ajuste_hora_Csaturacao_"+ NOME +".csv")
    

Pasta acessada:  Dados_FOTOAGUA/Dados_brutos/LR
Tipo analisado:  LR
Pasta acessada:  Dados_FOTOAGUA/Dados_brutos/PV1
Tipo analisado:  PV1


#  Exclusão dados que excedem a saturação

In [10]:
for n in range(len(arqs)):
    print("Pasta acessada: ",arqs[n])
    Arquivo = arqs[n]
    NOME = Arquivo[28:]
    print("Tipo analisado: ", NOME)
    result_OD = pd.read_csv(Arquivo + "/UNIAO/Ajuste_hora_Csaturacao_"+ NOME +".csv",sep=',',index_col=0);
    ################################################################################## 
    n = result_OD['OD(mg/L)'].argmax()
    df_mask = result_OD['Saturação OD (%)']<100
    filtered_df = result_OD[df_mask]
    filtered_df
    n = filtered_df['OD(mg/L)'].argmax()
    ##################################################################################
    filtered_df.index = pd.to_datetime(filtered_df.index)
    filtered_df = filtered_df.resample('10min').mean()
    filtered_df.to_csv("Dados Tratados/Ajuste_hora_Csaturacao_sem_excedentes_"+ NOME +".csv")
    

Pasta acessada:  Dados_FOTOAGUA/Dados_brutos/LR
Tipo analisado:  LR
Pasta acessada:  Dados_FOTOAGUA/Dados_brutos/PV1
Tipo analisado:  PV1


# DADOS MINIPAR

In [191]:
##########################################################################
##########################################################################
def arruma_PAR(ARQ):
    ler = open(ARQ + ".txt", "r")
    N = ler.readlines()
    
    ler.close()
    y = ARQ[39:]
    x = ARQ[:-18]
    #print(x + "AUX/" + y +".aux")
    esc = open(x + "AUX/" + y +".aux", "w")
    esc.write('Segundos,PAR (umol/(s m^2))\n')
    for I in range(len(N)):
        A = re.split(r"[,]", N[I])  
        #print(len(A))
        if len(A) >= 7:
            esc.write("{0},{1}\n".format(A[0], A[3]))

    esc.close()
##########################################################################
##########################################################################

In [192]:
caminho = "Dados_FOTOAGUA/Dados_brutos/miniPAR/"
arqs=[
    caminho + 'LR',
    caminho + 'PV1'
]

In [193]:
for n in range(len(arqs)):
    print("Pasta acessada: ",arqs[n])
    Arquivo = arqs[n]
    NOME = Arquivo[28:]
    print("Tipo analisado: ", NOME)
    ##################################################################################
    (Path('.') / 'Dados Tratados/').mkdir(parents=True, exist_ok=True)# cria pasta para o Ano analisado
    (Path('.') / Arquivo /"AUX").mkdir(parents=True, exist_ok=True)# cria pasta para o Ano analisado
    (Path('.') / Arquivo /"UNIAO").mkdir(parents=True, exist_ok=True)# cria pasta para o Ano analisado
    super_caminhos = []
    for item in os.listdir(arqs[n]):
        if not item.startswith('.') and os.path.isfile(os.path.join(arqs[n], item)):
            super_caminhos.append(item)
    arquivos_lidos = super_caminhos
    print("Quantidade de arquivos: ",len(arquivos_lidos))  
    
    result_miniPAR = pd.DataFrame()
    i = 0
    for k in range(len(arquivos_lidos)):
        nome = arquivos_lidos[k]
        nome = nome[:-4]
        #print(arqs[n]+"/"+nome)
        arruma_PAR(arqs[n]+"/"+nome)
        dt_PAR = pd.read_csv(arqs[n]+"/AUX/" +nome + ".aux", sep=',');
        nome = nome[:-7]
        dt_PAR["Data"] = nome
    

        result_miniPAR = pd.concat([result_miniPAR, dt_PAR])
    print(len(result_miniPAR),"\n")
    result_miniPAR = result_miniPAR.sort_values(by=['Segundos'])
    ##################################################################################
    result_miniPAR.to_csv(Arquivo + "/UNIAO/"+ NOME[:7] +".csv",index=False)


Pasta acessada:  Dados_FOTOAGUA/Dados_brutos/miniPAR/LR
Tipo analisado:  miniPAR/LR
Quantidade de arquivos:  149
42423 

Pasta acessada:  Dados_FOTOAGUA/Dados_brutos/miniPAR/PV1
Tipo analisado:  miniPAR/PV1
Quantidade de arquivos:  147
42399 



In [196]:
for n in range(len(arqs)):
    print("Pasta acessada: ",arqs[n])
    Arquivo = arqs[n]
    NOME = Arquivo[28:]
    print("Tipo analisado: ", NOME)
    result_OD = pd.read_csv(Arquivo + "/UNIAO/"+ NOME[:7] +".csv",sep=',');
    ################################################################################## 
    result_OD = result_OD.sort_values(by=['Segundos'])
    #result_OD = result_OD.set_index('Data')
    cal = 0
    segundos = np.array(result_OD['Segundos'])
    segundos_ajustados = segundos - cal
    vetor_dia = np.zeros(len(result_OD))
    vetor_min = np.zeros(len(result_OD))
    vetor_hora = np.zeros(len(result_OD))
    vetor_ano = np.zeros(len(result_OD))
    vetor_month = np.zeros(len(result_OD))

    lista = []

    for k in range(len(result_OD)):
        objeto_data = dt.fromtimestamp(segundos_ajustados[k])
        data_br = objeto_data + timedelta(hours=0) # sem alterar a hora
        vetor_dia[k] = str(data_br.day)
        vetor_month[k] = str(data_br.month)
        vetor_ano[k] = str(data_br.year)
        vetor_hora[k] = str(data_br.hour)
        vetor_min[k] = str(data_br.minute)
        texto = str(data_br.year) + "-" + str(data_br.month) + "-" + str(data_br.day) + "  " + str(data_br.hour) + ":" + str(data_br.minute)
        lista.append(texto)
    result_OD["Data Calculada"] = lista   
    now = datetime.datetime.now()
    now = datetime.datetime.strftime(now, '%Y-%m-%dT %H:%M')
    now = datetime.datetime.now()
    now = datetime.datetime.strftime(now, '%Y-%m-%dT %H:%M')
    result_OD = result_OD.set_index('Data Calculada')
    ################################################################################## 
    result_OD.index = pd.to_datetime(result_OD.index)
    result_OD = result_OD.resample('10min').mean()
    result_OD.to_csv("Dados Tratados/Ajuste_hora_"+ NOME[:7]+ "_"+ NOME[8:] +".csv")

Pasta acessada:  Dados_FOTOAGUA/Dados_brutos/miniPAR/LR
Tipo analisado:  miniPAR/LR


/var/folders/b1/jt5k1wqd6456ym1lhh704g000000gn/T/ipykernel_4817/1303371847.py:39: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  result_OD = result_OD.resample('10min').mean()


Pasta acessada:  Dados_FOTOAGUA/Dados_brutos/miniPAR/PV1
Tipo analisado:  miniPAR/PV1


/var/folders/b1/jt5k1wqd6456ym1lhh704g000000gn/T/ipykernel_4817/1303371847.py:39: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  result_OD = result_OD.resample('10min').mean()


In [197]:
result_OD

,Segundos,PAR (umol/(s m^2))
Data Calculada,,
2023-03-23 10:10:00,1.679578e+09,27.60
2023-03-23 10:20:00,1.679578e+09,26.45
2023-03-23 10:30:00,1.679579e+09,29.00
2023-03-23 10:40:00,1.679579e+09,34.95
2023-03-23 10:50:00,1.679580e+09,50.25
...,...,...
2023-08-17 14:50:00,1.692295e+09,9.25
2023-08-17 15:00:00,1.692296e+09,9.05
2023-08-17 15:10:00,1.692296e+09,9.25


# Concatenando os dados miniPAR + miniDOT

In [207]:
result_LR_OD = pd.read_csv("Dados Tratados/Ajuste_hora_Csaturacao_sem_excedentes_LR.csv",sep=',',index_col=0);
result_LR_OD = result_LR_OD.drop(['Segundos', 'OD de Saturação (mg/L)'], axis=1)
result_LR_OD = result_LR_OD.rename(columns={'OD(mg/L)': 'OD(mg/L) LR', 'Saturação OD (%)': 'Saturação OD (%) LR', 'Temperatura(deg C)': 'Temperatura(deg C) LR'})
#
result_PV1_OD = pd.read_csv("Dados Tratados/Ajuste_hora_Csaturacao_sem_excedentes_PV1.csv",sep=',',index_col=0);
result_PV1_OD = result_PV1_OD.drop(['Segundos', 'OD de Saturação (mg/L)'], axis=1)
result_PV1_OD = result_PV1_OD.rename(columns={'OD(mg/L)': 'OD(mg/L) PV1', 'Saturação OD (%)': 'Saturação OD (%) PV1', 'Temperatura(deg C)': 'Temperatura(deg C) PV1'})


result_LR_PAR = pd.read_csv("Dados Tratados/Ajuste_hora_miniPAR_LR.csv",sep=',',index_col=0);
result_LR_PAR = result_LR_PAR.drop(['Segundos'], axis=1)
result_LR_PAR = result_LR_PAR.rename(columns={'PAR (umol/(s m^2))': 'PAR (umol/(s m^2)) LR'})


result_PV1_PAR = pd.read_csv("Dados Tratados/Ajuste_hora_miniPAR_PV1.csv",sep=',',index_col=0);
result_PV1_PAR = result_PV1_PAR.drop(['Segundos'], axis=1)
result_PV1_PAR = result_PV1_PAR.rename(columns={'PAR (umol/(s m^2))': 'PAR (umol/(s m^2)) PV1'})


m = pd.merge(result_LR_OD, result_PV1_OD, how = 'inner', left_index=True, right_index=True)
m = pd.merge(m, result_LR_PAR, how = 'inner', left_index=True, right_index=True)
m = pd.merge(m, result_PV1_PAR, how = 'inner', left_index=True, right_index=True)

################################################################################## 
m.to_csv("Dados Tratados/Dados_concatenados.csv")
    